In [4]:
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import string
import re
import unicodedata

import gensim
from gensim.corpora.dictionary import Dictionary
from gensim.models import LdaModel
from gensim.models.coherencemodel import CoherenceModel
import itertools
import tqdm

from pprint import pprint

import pyLDAvis
import pyLDAvis.gensim

import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)


# Download necessary NLTK data
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

/Users/mavin/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
[nltk_data] Downloading package punkt to /Users/mavin/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/mavin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/mavin/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [5]:
df = pd.read_csv("./web_of_sicence_dataset/wof_ds_papers.csv")
df.head(5)

,Article Title,Abstract,Publication Date,Publication Year,DOI
0,Determinants of digital well-being,How can people lead fulfilling lives both than...,2024 SEP 20,2024,10.1007/s00146-024-02071-2
1,AI-based lumbar central canal stenosis classif...,ObjectivesThe assessment of lumbar central can...,2024 SEP 20,2024,10.1007/s00330-024-11080-0
2,"Analysis of Depression, Anxiety, Stress Scale ...",This study employs advanced data mining techni...,2024 SEP 19,2024,10.1111/ejed.12778
3,An algorithm for cattle counting in rangeland ...,To effectively address common issues such as c...,2024 SEP 19,2024,10.1049/ipr2.13240
4,Game Theoretic Approach Toward Detection of In...,Numerous researchers have scrutinized evasive ...,2024 SEP 19,2024,10.1002/spy2.467


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7842 entries, 0 to 7841
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Article Title     7842 non-null   object
 1   Abstract          7823 non-null   object
 2   Publication Date  7748 non-null   object
 3   Publication Year  7842 non-null   int64 
 4   DOI               7841 non-null   object
dtypes: int64(1), object(4)
memory usage: 306.5+ KB


### Text Processing

In [7]:
df = df.dropna()
df = df.drop_duplicates()

In [8]:
def normalize_text_for_bow(text, custom_stopwords=None):
    # Convert to lowercase
    text = text.lower()
    
    # Unicode normalization
    text = unicodedata.normalize('NFKD', text)
    
    # Remove special characters
    text = re.sub(r'[^\w\s]', '', text)
    
    # Tokenize
    tokens = word_tokenize(text)
    
    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    if custom_stopwords:
        stop_words.update(custom_stopwords)
    tokens = [token for token in tokens if token not in stop_words]
    
    # Remove short words (optional, adjust as needed)
    tokens = [token for token in tokens if len(token) > 2]
    
    return tokens

In [9]:
custom_stopwords = [
    # General research terms
    'study', 'research', 'paper', 'article', 'journal',
    'analysis', 'method', 'methodology', 'approach',
    'result', 'results', 'finding', 'findings',
    'conclusion', 'conclusions', 'discussion',
    'hypothesis', 'hypotheses', 'theory', 'theoretical',
    'experiment', 'experimental', 'observation', 'observations',

    # Data and statistics
    'data', 'dataset', 'sample', 'population',
    'variable', 'variables', 'factor', 'factors',
    'significant', 'significance', 'correlation',
    'analysis', 'regression', 'model', 'modeling',

    # Publication-related
    'abstract', 'introduction', 'background', 'literature',
    'review', 'method', 'methods', 'materials',
    'procedure', 'procedures', 'protocol', 'protocols',
    'published', 'unpublished', 'preprint',
    'journal', 'publication', 'doi', 'isbn',

    # Common academic phrases
    'et', 'al', 'ie', 'eg', 'cf',
    'respectively', 'thus', 'hence', 'therefore',
    'however', 'moreover', 'furthermore', 'additionally',

    # Time-related
    'year', 'month', 'week', 'day',
    'annual', 'quarterly', 'monthly', 'daily',

    # Measurement and quantity
    'measure', 'measurement', 'quantity', 'amount',
    'level', 'levels', 'degree', 'degrees',
    'rate', 'rates', 'percentage', 'percentages',

    # Generic descriptors
    'high', 'low', 'medium', 'average',
    'large', 'small', 'significant', 'insignificant',

    # Common verbs
    'show', 'shows', 'shown', 'indicate', 'indicates', 'indicated',
    'suggest', 'suggests', 'suggested', 'report', 'reports', 'reported',
    'observe', 'observes', 'observed', 'find', 'finds', 'found',
    'note', 'notes', 'noted', 'present', 'presents', 'presented',

    # Research process
    'collect', 'collected', 'analyze', 'analyzed',
    'investigate', 'investigated', 'examine', 'examined',
    'evaluate', 'evaluated', 'assess', 'assessed',

    # Miscellaneous
    'based', 'using', 'used', 'via', 'through',
    'within', 'between', 'among', 'across',
    'figure', 'table', 'equation', 'section'
]

In [10]:
# Combine Title and Abstract
df['combined_text'] = df['Article Title'] + ' ' + df['Abstract'].fillna('')



# Apply preprocessing to combined text (pass custom stopwords if needed)
df['normalized_tokens'] = df['combined_text'].apply(lambda x: normalize_text_for_bow(x, custom_stopwords))

### Create Bag of Words Representation

In [11]:
texts = df['normalized_tokens']

# Step 1: Create a dictionary representation of the documents
dictionary = Dictionary(texts)

In [12]:
# Step 2: Create a Bag of Words (BoW) corpus
corpus = [dictionary.doc2bow(text) for text in texts]

### LDA

In [13]:
# Build LDA model
lda_model = LdaModel(corpus=corpus,
                     id2word=dictionary,
                     num_topics=20,
                     random_state=0,
                     chunksize=100,
                     alpha='asymmetric',
                     eta=0.1,
                     passes=20,
                     per_word_topics=True)

pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.055*"language" + 0.031*"text" + 0.018*"examination" + 0.014*"bias" + '
  '0.013*"emotional" + 0.013*"structured" + 0.012*"llm" + 0.010*"worker" + '
  '0.010*"semantic" + 0.009*"writing"'),
 (1,
  '0.028*"learning" + 0.016*"algorithm" + 0.016*"network" + 0.015*"machine" + '
  '0.012*"feature" + 0.012*"image" + 0.012*"performance" + 0.012*"deep" + '
  '0.010*"proposed" + 0.010*"accuracy"'),
 (2,
  '0.025*"gene" + 0.022*"cell" + 0.012*"drug" + 0.011*"expression" + '
  '0.011*"layer" + 0.010*"protein" + 0.010*"annotation" + 0.009*"breast" + '
  '0.009*"bayesian" + 0.009*"differential"'),
 (3,
  '0.020*"change" + 0.018*"climate" + 0.017*"specie" + 0.017*"mental" + '
  '0.011*"area" + 0.011*"environmental" + 0.010*"global" + 0.009*"nurse" + '
  '0.009*"might" + 0.009*"region"'),
 (4,
  '0.039*"patient" + 0.017*"disease" + 0.014*"risk" + 0.013*"treatment" + '
  '0.012*"clinical" + 0.011*"group" + 0.011*"diagnosis" + 0.010*"age" + '
  '0.010*"outcome" + 0.009*"health"'),
 (5,
  '0.02

In [14]:
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=texts, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('nCoherence Score: ', coherence_lda)

nCoherence Score:  0.46715761228385977


In [15]:
#Creating Topic Distance Visualization 
pyLDAvis.enable_notebook()
p = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary)
p

/Users/mavin/opt/anaconda3/lib/python3.8/site-packages/pyLDAvis/_prepare.py:243: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.342847  0.006227       1        1  24.557006
9     -0.331533 -0.001306       2        1  21.471482
4     -0.191086 -0.251856       3        1   8.337794
6     -0.179775  0.125140       4        1   7.850582
14    -0.153348  0.273803       5        1   7.532954
17    -0.131489 -0.040972       6        1   6.062895
19    -0.139208 -0.179391       7        1   5.094488
18    -0.020531 -0.007407       8        1   3.852102
2     -0.015172  0.030363       9        1   3.603363
10     0.004680  0.101313      10        1   3.422489
3      0.026625  0.021435      11        1   2.703877
0      0.124584 -0.003435      12        1   1.433824
15     0.143650 -0.004415      13        1   1.015046
13     0.157626 -0.009497      14        1   0.765680
16     0.173416 -0.010036      15        1   0.598707
12     0.174560 -0.010000      16        1   0.402384
8      0.172953 -0.010018      17        1   0.393776
11     0.172571 -0.010273      18        1   0.386590
7      0.176202 -0.009921      19        1   0.298467
5      0.178122 -0.009754      20        1   0.216495, topic_info=                  Term         Freq        Total Category  logprob  loglift
333           learning  7965.000000  7965.000000  Default  30.0000  30.0000
177             system  4196.000000  4196.000000  Default  29.0000  29.0000
149            patient  3447.000000  3447.000000  Default  28.0000  28.0000
258            student  2586.000000  2586.000000  Default  27.0000  27.0000
87           algorithm  3909.000000  3909.000000  Default  26.0000  26.0000
...                ...          ...          ...      ...      ...      ...
3708              felt     2.951055     4.481031  Topic20  -6.6011   5.7177
2118       alleviating     2.802456     4.332586  Topic20  -6.6527   5.6997
3003              null     2.575295     4.106465  Topic20  -6.7373   5.6688
3967   appropriateness     1.839240     3.369184  Topic20  -7.0739   5.5300
12449       imputation    12.217152    33.761646  Topic20  -5.1804   5.1189

[896 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
6729      3  0.995312      0001
5466     18  0.880193       081
3431      7  0.994983       100
6739     11  0.994105      2020
8480     14  0.968665      95ci
...     ...       ...       ...
4847     12  0.991127    worker
3819      7  0.992826  workload
3459     16  0.945937     write
3413     12  0.986672   writing
1525     14  0.978519      xray

[1290 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 10, 5, 7, 15, 18, 20, 19, 3, 11, 4, 1, 16, 14, 17, 13, 9, 12, 8, 6])